# Bastion hosts

There are many reasons for using bastion hosts:

  - security access eg in cloud environment
  - vpn eg via windows hosts
  
The latter case is quite boring as ansible doesn't support windows as a client platform.

A standard approach is:

  - have a ssh server or a proxy installed on the bastion
  - connecto the bastion to the remote network (eg. via vpn)
  - configure ssh options in ansible to connect thru the bastion
  
We'll do this via two configuration files:

  - a standard ssh_config where we put the passthru configuration
  - a simple ansible.cfg referencing ssh_config
  
This approach allows us:

  1. to test the standard ssh connection thru the bastion without messing with ansible
  2. keep ansible.cfg simple in case we want to reuse them from the intranet (Eg. without traversing the bastion)
  
  

In [1]:
cd exercise-06/

/notebooks/exercise-06


## ssh_config

Instead of continuously passing options to ssh, we can use [`-F ssh_config`](/edit/notebooks/exercise-06/ssh_config) and put configurations there.


In [15]:
!cat ssh_config

#
# This is the ssh client configuration file.
#


# We can specify an ssh user/key for each network
# which is very useful on cloud deployments (eg. openstack)
Host 172.23.0.*
  User centos
  IdentityFile ~/id_ansible
  # Just for testing purposes, we could 
  # want to skip host key checking
  StrictHostKeyChecking no

# And can alias a node and *eventually* set a custom port
Host bastion
  User root
  Hostname 172.17.0.5
  StrictHostKeyChecking no
  UserKnownHostsFile /dev/null
  IdentityFile ../exercise-01/id_ansible  

#
# Here's the magic ProxyCommand stuff.
# BEWARE: some ssh-client versions re-process bastion to 172.17.0.5
#         and this would end in a recursive proxy transit. Always
#         set an EXCLUSION for your bastion host ip.
# 
Host !172.17.0.5,172.17.0.*
  User root
  StrictHostKeyChecking no
  UserKnownHostsFile /dev/null
  IdentityFile ../exercise-01/id_ansible
  ProxyCommand ssh -F ssh_config root@bastion -W %h:%p



If we don't use it, we can turn off `GSSApiAuthentication` which attempts may slow down the connection.

## Connect to the bastion

Test connectivity to the bastion. Check your host ips and modify  [`ssh_config`](/edit/notebooks/exercise-06/ssh_config)  accordingly.

** Replace ALL bastion occurrencies, including the one below the BEWARE note**

In [3]:
fmt=r'{{.NetworkSettings.IPAddress}}'
!docker  -H tcp://172.17.0.1:2375 inspect  ansible101_bastion_1 --format {fmt} # pass variables *before* commands ;)

172.17.0.5


### Exercise

Write the [ssh-copy-id.yml](/edit/notebooks/exercise-06/ssh-copy-id.yml) playbook to install an ssh key to the bastion.


Bastion credentials are:

  - user: `root`
  - password `root`
 
Try to do it without watching the previous exercises:

  - modify the empty [`ansible.cfg`](/edit/notebooks/exercise-06/ansible.cfg)
      * referencing a pin file 
      * passing [`[ssh_connection]`](http://docs.ansible.com/ansible/intro_configuration.html#openssh-specific-settings) arguments to avoid ssh key mismatches
      * pointing to the local inventory
  - store credentials in the encrypted [`vault.yml`](/edit/notebooks/exercise-06/vault.yml). 
  - provide an [`inventory`](/edit/notebooks/exercise-06/inventory) file 

You can reuse the old id_ansible key or:

  - create a new one and adjust the reference in [` ssh_config`](/edit/notebooks/exercise-06/ssh_config) 
  
Hint:

  - if you provide an IdentityFile, password authentication won't work on the `bastion` node;
  - you *must* copy ssh id file using password authentication and eventually clean up your known_host file

In [8]:
# Use this cell to create the pin file and then encrypt the vault

In [ ]:
# Use this cell to test/run  the playbook. You can --limit the execution to the bastion host only.

In [14]:
!ssh -Fssh_config bastion hostname

cb6a5b6f0620


# ansible.cfg and ssh_config

In the previous exercise, we used the `[ssh_connection]` stanza to configure ssh connections.

We can instead just set

```
[ssh_connection]
ssh_args = -F ssh_config
```

Write everything in ssh_config.


Connecting via bastion in ansible enforcing multiple references to ssh_config

### Exercise

Uncomment the last lines of  [` ssh_config`](/edit/notebooks/exercise-06/ssh_config) and try to use bastion for connecting to the other hosts
  

In [17]:
fmt=r'{{.NetworkSettings.IPAddress}}'
!docker  -H tcp://172.17.0.1:2375 inspect  ansible101_web_1 --format {fmt} # pass variables *before* commands ;)

172.17.0.4


In [25]:
!ssh -F ssh_config  root@172.17.0.4 ip -4 -o a  # get host ip

1: lo    inet 127.0.0.1/8 scope host lo\       valid_lft forever preferred_lft forever
44: eth0    inet 172.17.0.4/16 scope global eth0\       valid_lft forever preferred_lft forever


## Exercise

Configure your `ansible.cfg` so that every web host is accessed via the bastion.

  - recycle your dynamic inventory script to access web hosts
  - your id_ansible key should already be on your web hosts 
  - use ansible -m ping to check host connectivity
  - run `ps -ef | grep ssh` on your docker host to check all the `ProxyCommand` processes.